In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm

from statsmodels.regression.linear_model import OLS

In [54]:
df = pd.read_csv("Data.csv")

In [19]:
def df_OLS_anal(df: pd.DataFrame):
    max = len(df)
    for index, row in df.iterrows():
        if index < max - 1 and abs(df.loc[index, 'Time'] - df.loc[index + 1, 'Time']) < 1e-6:
            # print(index)
            df.loc[index + 1, 'Volume'] = df.loc[index + 1, 'Volume'] + df.loc[index, 'Volume']
            df.loc[index + 1, 'AskBefore'] = df.loc[index, 'AskBefore']
            df = df.drop([index])

    deltaA = df["AskAfter"][1:] - df["AskBefore"][1:]
    deltat = np.diff(df["Time"])
    x = df["Volume"][1:]

    dA = deltaA / deltat
    dx = x / deltat

    ddx = np.diff(dx)
    ddA = np.diff(dA)

    OLSdf = pd.DataFrame({
        "SUM": ddA,
        "-rho": deltaA[:-1],
        "rho lambda": x[:-1],
        "kappa + lambda": ddx
    })

    Regressand = OLSdf["SUM"]

    Regressors = OLSdf[["-rho", "rho lambda", "kappa + lambda"]]

    Regressors = sm.add_constant(Regressors, prepend=False)

    model = OLS(Regressand, Regressors)

    res = model.fit()

    print(res.summary()) 

In [47]:
def df_OLS_coefs(df: pd.DataFrame):
    max = len(df)
    for index, row in df.iterrows():
        if index < max - 1 and abs(df.loc[index, 'Time'] - df.loc[index + 1, 'Time']) < 1e-6:
            # print(index)
            df.loc[index + 1, 'Volume'] = df.loc[index + 1, 'Volume'] + df.loc[index, 'Volume']
            df.loc[index + 1, 'AskBefore'] = df.loc[index, 'AskBefore']
            df = df.drop([index])

    deltaA = df["AskAfter"][1:] - df["AskBefore"][1:]
    deltat = np.diff(df["Time"])
    x = df["Volume"][1:]

    dA = deltaA / deltat
    dx = x / deltat

    ddx = np.diff(dx)
    ddA = np.diff(dA)

    OLSdf = pd.DataFrame({
        "SUM": ddA,
        "-rho": deltaA[:-1],
        "rho lambda": x[:-1],
        "kappa + lambda": ddx
    })

    Regressand = OLSdf["SUM"]

    Regressors = OLSdf[["-rho", "rho lambda", "kappa + lambda"]]

    Regressors = sm.add_constant(Regressors, prepend=False)

    model = OLS(Regressand, Regressors)

    res = model.fit()

    rho = - res.params[0]
    lamb = res.params[1] / rho
    kappa = res.params[2] - lamb

    return rho, kappa, lamb

In [48]:
df_OLS_coefs(df)

/tmp/ipykernel_13799/3114995880.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  rho = - res.params[0]
/tmp/ipykernel_13799/3114995880.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lamb = res.params[1] / rho
/tmp/ipykernel_13799/3114995880.py:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  kappa = res.params[2] - lamb


(15033.62964678484, -5.048376251178723e-08, 5.707670757249037e-08)

In [368]:
df_OLS_anal(df)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     144.3
Date:                Sun, 14 Jan 2024   Prob (F-statistic):           1.74e-91
Time:                        14:25:56   Log-Likelihood:                -90059.
No. Observations:                9743   AIC:                         1.801e+05
Df Residuals:                    9739   BIC:                         1.802e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho           -1.503e+04    981.693    -15.

In [55]:
df = df[df["Time"] < 6200]
df = df[df["Time"] > 5800]
df = df.reset_index()
df = df.drop(["index"], axis=1)
df_OLS_anal(df)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.233
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     22.85
Date:                Tue, 16 Jan 2024   Prob (F-statistic):           5.85e-13
Time:                        16:30:10   Log-Likelihood:                -2002.7
No. Observations:                 230   AIC:                             4013.
Df Residuals:                     226   BIC:                             4027.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho           -1.263e+04   5073.805     -2.

In [374]:
df = df[df["AskAfter"] - df["AskBefore"] > 0.01]
df = df.reset_index()
df = df.drop(["index"], axis=1)
df_OLS_anal(df)

                            OLS Regression Results                            
Dep. Variable:                    SUM   R-squared:                       0.439
Model:                            OLS   Adj. R-squared:                  0.436
Method:                 Least Squares   F-statistic:                     159.5
Date:                Sun, 14 Jan 2024   Prob (F-statistic):           2.17e-76
Time:                        14:28:20   Log-Likelihood:                -6133.8
No. Observations:                 616   AIC:                         1.228e+04
Df Residuals:                     612   BIC:                         1.229e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
-rho           -4099.6975   2230.535     -1.